In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import plotly.express as px
import pycountry
from collections import defaultdict
import requests

# Load Verra data 

In [53]:
r = requests.post(
    'https://registry.verra.org/uiapi/asset/asset/search?$maxResults=2000&$count=true&$skip=0&format=csv',
    json={"program":"VCS","issuanceTypeCodes":['ISSUE']}
)
rename_map = {
 'issuanceDate':'Issuance Date', 
 'programObjectives': 'Sustainable Development Goals', 
 'instrumentType': 'Credit Type', 
 'vintageStart':'Vintage Start',
 'vintageEnd':'Vintage End', 
 'reportingPeriodStart':'Reporting Period Start', 
 'reportingPeriodEnd':'Reporting Period End',
 'resourceIdentifier':'ID', 
 'resourceName':'Name', 
 'region':'Region', 
 'country':'Country',
 'protocolCategory': 'Project Type', 
 'protocol':'Methodology', 
 'totalVintageQuantity':'Total Vintage Quantity', 
 'quantity':'Quantity Issued',
 'serialNumbers':'Serial Number', 
 'additionalCertifications': 'Additional Certifications', 
 'retiredCancelled':'Is Cancelled',
 'retireOrCancelDate':'Retirement/Cancellation Date', 
 'retirementBeneficiary':'Retirement Beneficiary', 
 'retirementReason':'Retirement Reason',
 'retirementDetails':'Retirement Details', 
 'inputTypes': 'Input Type', 
 'holdingIdentifier': 'Holding ID'
}
vcus = pd.DataFrame(r.json()['value']).rename(columns=rename_map)

# Fixing up datetimes
vcus['Vintage Start'] = pd.to_datetime(vcus['Vintage Start'])
vcus['Vintage End'] = pd.to_datetime(vcus['Vintage End'])
vcus['Issuance Date'] = pd.to_datetime(vcus['Issuance Date'])
vcus['Retirement/Cancellation Date'] = pd.to_datetime(vcus['Retirement/Cancellation Date'])
# Assgining vintage year
vcus['Vintage'] = vcus['Vintage Start'].dt.year
vcus['Issuance Year'] = vcus['Issuance Date'].dt.year
# Assigning retirement age
vcus['Days to Retirement'] = (vcus['Retirement/Cancellation Date'] - vcus['Issuance Date']).dt.days
# Assigning staleness if not retired
now = dt.datetime.now()
vcus.loc[vcus['Days to Retirement'].isna(),'Staleness'] = (now-vcus[vcus['Days to Retirement'].isna()]['Issuance Date']).dt.days
# Assigning retirement status 
vcus['Status'] = ['Retired' if row['Days to Retirement']>0 else 'Available' for i, row in vcus.iterrows()]
# Some nice renaming since these are too verbose
vcus = vcus.rename(columns={'Total Vintage Quantity': 'Vintage Quantity', 'Quantity Issued': 'Quantity'})
# Toucan retirement credits
vcus.loc[vcus['Retirement Details'].str.contains('TOUCAN').fillna(False), 'Toucan'] = True
vcus['Toucan'] = vcus['Toucan'].fillna(False)
vcus.loc[vcus['Toucan'], 'Toucan Bridger'] = "https://polygonscan.com/address/"+vcus.query('Toucan')['Retirement Beneficiary']
# Toucan AM0001 methodology blacklist
toucan_blacklist_date = dt.datetime(year=2021, month=12, day=13)
vcus['Toucan Blacklisted'] = False
vcus.loc[np.logical_and(vcus['Retirement/Cancellation Date']>=toucan_blacklist_date, vcus['Methodology']=="AM0001"),'Toucan Blacklisted'] = True
# Building up an index for country codes
vcus['Country'] = vcus['Country'].replace('South Korea', 'Korea, Republic of')
country_index = defaultdict(str,{country:pycountry.countries.search_fuzzy(country)[0].alpha_3 for country in vcus.Country.astype(str).unique() if country!='nan'})
# ... so we could have country codes for visualizations
vcus['Country Code'] = [country_index[country] for country in vcus['Country']]
# Are vintage batches ever larger than 1 year?
vcus['Vintaged Batch Years'] = (vcus['Vintage End']-vcus['Vintage Start']).dt.days/365

# Relevant day stuff
last_time = dt.datetime.combine(dt.date.today(), dt.datetime.min.time())
seven_day_start = last_time - dt.timedelta(days=7)
last_seven_day_start = seven_day_start - dt.timedelta(days=7)
thirty_day_start = last_time - dt.timedelta(days=30)
last_thirty_day_start = thirty_day_start - dt.timedelta(days=30)
tpool = vcus.query('Toucan and ~`Toucan Blacklisted`')

# Seven day pool subsets    
sd_pool = tpool.query("(`Retirement/Cancellation Date`>=@seven_day_start) and (`Retirement/Cancellation Date`<@last_time)")
last_sd_pool = tpool.query("(`Retirement/Cancellation Date`>=@last_seven_day_start) and (`Retirement/Cancellation Date`<@seven_day_start)")
# Thirty day pool subsets
td_pool = tpool.query("(`Retirement/Cancellation Date`>=@thirty_day_start) and (`Retirement/Cancellation Date`<@last_time)")
last_td_pool = tpool.query("(`Retirement/Cancellation Date`>=@last_thirty_day_start) and (`Retirement/Cancellation Date`<@thirty_day_start)")

In [55]:
vcus.columns

Index(['Issuance Date', 'Sustainable Development Goals', 'Credit Type',
       'Vintage Start', 'Vintage End', 'Reporting Period Start',
       'Reporting Period End', 'ID', 'Name', 'Region', 'Country',
       'Project Type', 'Methodology', 'Vintage Quantity', 'Quantity',
       'Serial Number', 'Additional Certifications', 'Is Cancelled',
       'Retirement/Cancellation Date', 'Retirement Beneficiary',
       'Retirement Reason', 'Retirement Details', 'Input Type', 'Holding ID',
       'Vintage', 'Issuance Year', 'Days to Retirement', 'Staleness', 'Status',
       'Toucan', 'Toucan Bridger', 'Toucan Blacklisted', 'Country Code',
       'Vintaged Batch Years'],
      dtype='object')

In [56]:
vcus[['Project Type', 'Methodology']]

,Project Type,Methodology
0,Energy industries (renewable/non-renewable sou...,ACM0002
1,Agriculture Forestry and Other Land Use,AR-ACM0003
2,Energy industries (renewable/non-renewable sou...,ACM0002
3,Energy industries (renewable/non-renewable sou...,ACM0002
4,Energy industries (renewable/non-renewable sou...,AMS-I.E.
...,...,...
159881,Energy industries (renewable/non-renewable sou...,AMS-I.D.
159882,Energy industries (renewable/non-renewable sou...,AMS-I.D.
159883,Energy industries (renewable/non-renewable sou...,AMS-I.D.
159884,Energy industries (renewable/non-renewable sou...,AMS-I.D.


In [57]:
vcus.groupby('Project Type')['Project Type'].count()

Project Type
Agriculture Forestry and Other Land Use                                                                                      98023
Agriculture Forestry and Other Land Use; Energy industries (renewable/non-renewable sources); Waste handling and disposal        1
Chemical industry                                                                                                              749
Chemical industry; Energy industries (renewable/non-renewable sources)                                                          92
Construction; Energy industries (renewable/non-renewable sources)                                                              250
Energy demand                                                                                                                  897
Energy demand; Energy industries (renewable/non-renewable sources)                                                             136
Energy demand; Waste handling and disposal                            

In [58]:
vcus.head()

,Issuance Date,Sustainable Development Goals,Credit Type,Vintage Start,Vintage End,Reporting Period Start,Reporting Period End,ID,Name,Region,...,Vintage,Issuance Year,Days to Retirement,Staleness,Status,Toucan,Toucan Bridger,Toucan Blacklisted,Country Code,Vintaged Batch Years
0,2022-04-22,None,VCU,2020-01-01,2020-12-31,2020-01-01,2020-12-31,1987,GENNEIA WIND PROJECTS IN ARGENTINA,Latin America,...,2020,2022,NaN,0.0,Available,False,NaN,False,ARG,1.000000
1,2022-04-22,None,VCU,2016-01-13,2016-12-31,2016-01-13,2016-12-31,2498,Afforestation of degraded grasslands in Caazap...,Latin America,...,2016,2022,NaN,0.0,Available,False,NaN,False,PRY,0.967123
2,2022-04-22,None,VCU,2018-01-01,2018-03-31,2018-01-01,2018-03-31,536,Uluabat Hydroelectric Power Plant,Middle East,...,2018,2022,NaN,0.0,Available,False,NaN,False,TUR,0.243836
3,2022-04-22,None,VCU,2017-01-01,2017-12-31,2017-01-01,2017-12-31,536,Uluabat Hydroelectric Power Plant,Middle East,...,2017,2022,NaN,0.0,Available,False,NaN,False,TUR,0.997260
4,2022-04-22,None,VCU,2015-01-01,2015-12-31,2015-01-01,2015-12-31,54,Cenol and Telha Forte Ceramics Switching Fuel ...,Latin America,...,2015,2022,NaN,0.0,Available,False,NaN,False,BRA,0.997260


# Additional Certifications flag

In [59]:
vcus['Additional Certifications'].value_counts()

CCB-Gold                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              47024
Social Carbon                                                                                                                                                                                                                                                                                                                                                                                                                               

In [60]:
vcus['has_ac'] = ~vcus['Additional Certifications'].isnull()

In [61]:
vcus.groupby('has_ac')['Quantity'].sum()

has_ac
False    595437335
True     311556281
Name: Quantity, dtype: int64

# Label retired vs. unretired credits

In [62]:
vcus['Retirement/Cancellation Date'].isnull().value_counts()

False    146802
True      13084
Name: Retirement/Cancellation Date, dtype: int64

In [70]:
rets = vcus[~vcus['Retirement/Cancellation Date'].isnull()]

In [71]:
rets['Retirement/Cancellation Date'].isnull().value_counts()

False    146802
Name: Retirement/Cancellation Date, dtype: int64

# Label tokenized credits

In [86]:
rets = rets[~rets['Retirement Details'].str.contains('TOUCAN|C3T').fillna(False)]

# Define Demand

### Features
- Categories (+ ACs)
- Delta btwn issuance and retirement
- Delta btwn issuance and vintage
- Delta btwn vintage and retirement
- % of blank retirements
- % of issued credits retired (by project)

Compare % of issued vs. retired against baseline avg for all of Verra

In [93]:
rets['issue_retire_delta'] = rets['Retirement/Cancellation Date'] - rets['Issuance Date']

In [88]:
rets['vintage_retire_delta'] = rets['Retirement/Cancellation Date'] - rets['Vintage Start']

In [94]:
rets['vintage_issuance_delta'] = rets['Issuance Date'] - rets['Vintage Start']

In [91]:
rets['issue_retire_delta'].value_counts()

71 days      1329
1 days       1171
320 days      940
70 days       917
72 days       890
             ... 
3842 days       1
3766 days       1
2308 days       1
3716 days       1
2883 days       1
Name: issue_retire_delta, Length: 3533, dtype: int64

In [92]:
rets['vintage_retire_delta'].value_counts()

1206 days    2132
1207 days    1748
1208 days    1642
2301 days    1090
1209 days     791
             ... 
6213 days       1
6708 days       1
4812 days       1
4755 days       1
5087 days       1
Name: vintage_retire_delta, Length: 5538, dtype: int64

In [97]:
rets['vintage_issuance_delta'].value_counts()

2303 days    5126
1923 days    3577
3644 days    3525
1129 days    3415
1135 days    2947
             ... 
766 days        1
4733 days       1
4597 days       1
1309 days       1
145 days        1
Name: vintage_issuance_delta, Length: 3267, dtype: int64

In [98]:
rets['is_blank'] = rets['Retirement Beneficiary'].isnull()

In [100]:
rets['is_blank'].value_counts()

False    73238
True     72654
Name: is_blank, dtype: int64